<a href="https://colab.research.google.com/github/BenWhann/ACME-Insurance-ML/blob/main/ACME_Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Problem Statememt
# QUESTION: ACME Insurance Inc. offers affordable health insurance to thousands of customer all over the United States. As the lead data scientist at ACME, you're tasked with creating an automated system to estimate
# the annual medical expenditure for new customers, using information such as their age, sex, BMI, children, smoking habits and region of residence.
# Estimates from your system will be used to determine the annual insurance premium (amount paid every month) offered to the customer. Due to regulatory requirements, you must be able to explain why your system outputs
# a certain prediction.
# You're given a CSV file containing verified historical data, consisting of the aforementioned information and the actual medical charges incurred by over 1300 customers.

# My objective is to find a way to estimate the value of the charges column using values in the other columns. If I can do so with historical data, then I should be able to with new customers too,
# simply by asking for information like their age, sex, BMI, number of kids, smoking habits, and region.

medical_charges_url = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv'
from urllib.request import urlretrieve
urlretrieve(medical_charges_url, 'medical.csv')

!pip install numpy --quiet
!pip install pandas --quiet
!pip install plotly --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet
!pip install scikit-learn --quiet
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
%matplotlib inline
# %matplotlib notebook

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

medical_df = pd.read_csv('medical.csv')
medical_df.info()
medical_df.describe()

ageChargeCorr = medical_df.charges.corr(medical_df.age)
# Age vs Charges correlation

bmiChargeCorr = medical_df.charges.corr(medical_df.bmi)
# BMI vs Charges correlation

childrenChargeCorr = medical_df.charges.corr(medical_df.children)
# Children vs Charges correlation

print(f'Age vs Charges correlation: {ageChargeCorr}')
# weak positive corr
print(f'BMI vs Charges correlation: {bmiChargeCorr}')
# weak positive corr
print(f'Children vs Charges correlation: {childrenChargeCorr}')
# no correlation

# Converting categorical smoker data to numerical data
smoker_values = {'no': 0, 'yes': 1}
# provides a dictionary for the map function
medical_df['smoker_value'] = medical_df.smoker.map(smoker_values)
# map function
smokerChargeCorr = medical_df.charges.corr(medical_df.smoker_value)
# strong positive correlation

# completing same operations for sex category
sex_values = {'female': 0, 'male': 1}
medical_df['sex_value'] = medical_df.sex.map(sex_values)
sexChargeCorr = medical_df.charges.corr(medical_df.sex_value)

# completing same operations for region category, but in a different way
enc = preprocessing.OneHotEncoder()
enc.fit(medical_df[['region']])
one_hot = enc.transform(medical_df[['region']]).toarray()
medical_df[['northeast', 'northwest', 'southeast', 'southwest']] = one_hot
print(medical_df.head())

correlation_matrix = medical_df.corr(numeric_only=True)
print(correlation_matrix)

# sns.heatmap(correlation_matrix, annot=True, cmap='Blues')
# plt.title('Correlation Matrix for Medical Data')

# Smoker and Age have the strongest correlations with Charges

non_smoker_df = medical_df[medical_df.smoker == 'no']
smoker_df = medical_df[medical_df.smoker == 'yes']
figAgeCharges = px.scatter(non_smoker_df, x='age', y='charges', color='smoker', opacity=0.8, hover_data=['sex'], title='Age vs Charges')
# figAgeCharges.show()

ages = non_smoker_df.age
agesSmoker = smoker_df.age

# Standardizing data

numeric_cols = ['age', 'bmi', 'children']
cat_cols = ['smoker_value', 'sex_value', 'northeast', 'northwest', 'southeast', 'southwest']
scaler = StandardScaler()
scaler.fit(medical_df[numeric_cols])

print(scaler.mean_)
print(scaler.var_)

scaled_inputs = scaler.transform(medical_df[numeric_cols])
categorical_data = medical_df[cat_cols].values
print(scaled_inputs)

# Creating optimized regression model using the fit method. The fit method calculates the best fit line using the ordinary least squares method. This is better for smaller data sets

def rmse(predictions, targets):
  return np.sqrt(np.mean(np.square(targets - predictions)))

def overallModel():
  print('OVERALL MODEL')
  inputs, targets = medical_df[['age', 'bmi', 'children']], medical_df['charges']
  model = LinearRegression().fit(inputs, targets)
  predictions = model.predict(inputs)
  loss = rmse(targets, predictions)
  print('Coefficients ', model.coef_)
  print('Intercept ', model.intercept_)
  print("RMSE Loss: ", loss)

def nonSmokerModel():
  print('NON SMOKER MODEL')
  inputs, targets = non_smoker_df[['age', 'bmi', 'children', 'northeast', 'northwest', 'southeast', 'southwest']], non_smoker_df['charges']
  model = LinearRegression().fit(inputs, targets)
  predictions = model.predict(inputs)
  loss = rmse(targets, predictions)
  print('Coefficients ', model.coef_)
  print('Intercept ', model.intercept_)
  print("RMSE Loss: ", loss)

def smokerModel():
  print('SMOKER MODEL')
  inputs, targets = smoker_df[['age', 'bmi', 'children', 'northeast', 'northwest', 'southeast', 'southwest']], smoker_df['charges']
  model = LinearRegression().fit(inputs, targets)
  predictions = model.predict(inputs)
  loss = rmse(targets, predictions)
  print('Coefficients ', model.coef_)
  print('Intercept ', model.intercept_)
  print("RMSE Loss: ", loss)

overallModel()
nonSmokerModel()
smokerModel()

new_customer = [[28,30,2]]
scaled_new_customer = scaler.transform(new_customer)
print('SCALED CUST ', scaled_new_customer)

def comprehensiveModel():
  print('COMPREHENSIVE MODEL')
  inputs = np.concatenate((scaled_inputs, categorical_data), axis=1)
  targets = medical_df['charges']
  model = LinearRegression().fit(inputs, targets)
  predictions = model.predict(inputs)
  loss = rmse(targets, predictions)
  print("RMSE Loss: ", loss)
  print('PREDICTION ', model.predict([[-0.79795355,-0.10882659,0.75107928,1,0,0,1,0,0]]))
  weights_df = pd.DataFrame({
      'feature': np.append(numeric_cols + cat_cols, 1),
      'weight': np.append(model.coef_, model.intercept_)
  })
  print(weights_df.sort_values('weight', ascending=False))



comprehensiveModel()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
Age vs Charges correlation: 0.2990081933306476
BMI vs Charges correlation: 0.19834096883362895
Children vs Charges correlation: 0.06799822684790478
   age     sex     bmi  children smoker  ... sex_value  northeast  northwest  southeast  southwest
0   19  female  27.900         0    yes  ...         0        0.0        0.0        0.0        1.0
1   18    male  33.770         1     no  ...         1        0.0        0.0        1.0        0.0
2   28    male  33.000         3   

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

